# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
pip install scipy==1.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import pandas as pd
import pathlib
import os
import scipy
from scipy.misc import imread, imresize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [4]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
#tf.set_random_seed(30)
tf.random.set_seed(30)

In [5]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [6]:
%cd /content/drive/My Drive/Colab Notebooks/Project_data

/content/drive/My Drive/Colab Notebooks/Project_data


In [7]:
df = pd.read_csv("train.csv")
df.head()

,WIN_20180925_17_08_43_Pro_Left_Swipe_new;Left_Swipe_new;0
0,WIN_20180925_17_18_28_Pro_Left_Swipe_new;Left_...
1,WIN_20180925_17_18_56_Pro_Left_Swipe_new;Left_...
2,WIN_20180925_17_19_51_Pro_Left_Swipe_new;Left_...
3,WIN_20180925_17_20_14_Pro_Left_Swipe_new;Left_...
4,WIN_20180925_17_21_28_Pro_Left_Swipe_new;Left_...


In [8]:
#root_path = 'gdrive/My Drive/Colab Notebooks/Project_data'
#list(pathlib.Path(root_path))

In [9]:
train_file = '/content/drive/My Drive/Colab Notebooks/Project_data/train.csv'
val_file = '/content/drive/My Drive/Colab Notebooks/Project_data/val.csv'

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [10]:
train_doc = np.random.permutation(open(train_file).readlines())
val_doc = np.random.permutation(open(val_file).readlines())
batch_size = 40 #experiment with the batch size

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [11]:
#set the no of frames
x = 30
#set width of image
y = 120
#set height of image
z = 120
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,x)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    tmp_image = imresize(image,(y,z))
                    tmp_image = tmp_image/127.5-1

                    batch_data[folder,idx,:,:,0] = (tmp_image[:,:,0]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (tmp_image[:,:,1]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (tmp_image[:,:,2]) #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batches: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,x,y,z,3))  
            batch_labels = np.zeros((batch_size,5))  
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate over the frames/images
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                                        
                    tmp_image = imresize(image,(y,z))
                    tmp_image = tmp_image/127.5-1  
                    
                    batch_data[folder,idx,:,:,0] = (tmp_image[:,:,0])
                    batch_data[folder,idx,:,:,1] = (tmp_image[:,:,1])
                    batch_data[folder,idx,:,:,2] = (tmp_image[:,:,2])
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels   

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [11]:
curr_dt_time = datetime.datetime.now()
train_path = '/content/drive/My Drive/Colab Notebooks/Project_data/train'
val_path = '/content/drive/My Drive/Colab Notebooks/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 15 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 15


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [13]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here
model = Sequential()       
model.add(Conv3D(8,kernel_size=(3,3,3),input_shape=(30, 120, 120, 3),padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv3D(16, (3, 3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(32, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(64, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(128, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))      

# Flatten layer 
model.add(Flatten())

model.add(Dense(1000, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(500, activation='relu'))
model.add(Dropout(0.5))

#Softmax layer
model.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [14]:
optimiser = tf.keras.optimizers.Adam(learning_rate=0.001) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 30, 120, 120, 8)   656       
                                                                 
 batch_normalization (BatchN  (None, 30, 120, 120, 8)  32        
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 30, 120, 120, 8)   0         
                                                                 
 conv3d_1 (Conv3D)           (None, 30, 120, 120, 16)  3472      
                                                                 
 activation_1 (Activation)   (None, 30, 120, 120, 16)  0         
                                                                 
 batch_normalization_1 (Batc  (None, 30, 120, 120, 16)  64       
 hNormalization)                                        

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [15]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [19]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [17]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [18]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  This is separate from the ipykernel package so we can avoid doing imports until


Source path =  /content/drive/My Drive/Colab Notebooks/Project_data/train ; batch size = 40


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning:     `imread` is deprecated!
    `imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imread`` instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
    Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


Epoch 1/15
15/17 [=========================>....] - ETA: 8:34 - loss: 7.5521 - categorical_accuracy: 0.2983 Batches:  17 Index: 40


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: DeprecationWarning:     `imread` is deprecated!
    `imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imread`` instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
    Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


17/17 [==============================] - ETA: 0s - loss: 7.1089 - categorical_accuracy: 0.3213  Source path =  /content/drive/My Drive/Colab Notebooks/Project_data/val ; batch size = 40
Batches:  3 Index: 40

Epoch 1: saving model to model_init_2022-09-1119_46_19.528232/model-00001-7.10890-0.32127-2.14893-0.18000.h5
17/17 [==============================] - 4511s 281s/step - loss: 7.1089 - categorical_accuracy: 0.3213 - val_loss: 2.1489 - val_categorical_accuracy: 0.1800 - lr: 0.0010
Epoch 2/15
17/17 [==============================] - ETA: 0s - loss: 2.0730 - categorical_accuracy: 0.4322
Epoch 2: saving model to model_init_2022-09-1119_46_19.528232/model-00002-2.07296-0.43223-1.39654-0.51667.h5
17/17 [==============================] - 250s 16s/step - loss: 2.0730 - categorical_accuracy: 0.4322 - val_loss: 1.3965 - val_categorical_accuracy: 0.5167 - lr: 0.0010
Epoch 3/15
17/17 [==============================] - ETA: 0s - loss: 1.6037 - categorical_accuracy: 0.5232
Epoch 3: saving model t

**Conclusion:** With a batch size of 40 and 15 epochs, the training accuracy (categorical_accuracy) is 0.84 and validation accuracy (val_categorical_accuracy) is 0.25

**Experiment 2:** Conv3D with an image size of 140 X 140 and batch size 40, 15 epochs

In [12]:
# Different size of images experimentation

channels = 3 # RGB as 3D Convlution
classes = 5
x = 30
y = 120
z = 120 

def generator_new(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,x)] #list of image numbers for a video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,channels)) # x is the number of images, (y,z) is the final size of the input and 3 is the number of channels
            batch_labels = np.zeros((batch_size,classes)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate over the frames/images of a folder
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                  
                    Temp_img = imresize(image,(y,z)) 
                    Temp_img = Temp_img.mean(axis=-1,keepdims=1)  # Let us convert to the Grey scale
                    Temp_img = Temp_img/127.5-1 
                    
                    batch_data[folder,idx] = Temp_img #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels      
              

        
        if (len(folder_list) != batch_size*num_batches):
            print("Batches: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,x,y,z,channels)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,classes)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                   
                    Temp_img = imresize(image,(y,z))
                    Temp_img = Temp_img.mean(axis=-1,keepdims=1)  # Let us convert to the Grey scale 
                    Temp_img = Temp_img/127.5-1 #Normalize data
                    
                    batch_data[folder,idx] = Temp_img
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

In [20]:
#from keras.models import Sequential, Model
#from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
#from keras.layers.convolutional import Conv3D, MaxPooling3D
#from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
#from keras import optimizers

# Input all the images sequencial by building the layer with dropouts and batchnormalisation

model_2 = Sequential()       
model_2.add(Conv3D(8,kernel_size=(3,3,3),input_shape=(30, 120, 120, 3),padding='same'))
model_2.add(BatchNormalization())
model_2.add(Activation('relu'))

model_2.add(Conv3D(16, (3, 3, 3), padding='same'))
model_2.add(Activation('relu'))
model_2.add(BatchNormalization())
model_2.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_2.add(Conv3D(32, (2, 2, 2), padding='same'))
model_2.add(Activation('relu'))
model_2.add(BatchNormalization())
model_2.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_2.add(Conv3D(64, (2, 2, 2), padding='same'))
model_2.add(Activation('relu'))
model_2.add(BatchNormalization())
model_2.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_2.add(Conv3D(128, (2, 2, 2), padding='same'))
model_2.add(Activation('relu'))
model_2.add(BatchNormalization())
model_2.add(MaxPooling3D(pool_size=(2, 2, 2)))      

# Flatten layer 

model_2.add(Flatten())

model_2.add(Dense(1000, activation='relu'))
model_2.add(Dropout(0.5))

model_2.add(Dense(500, activation='relu'))
model_2.add(Dropout(0.5))

#Softmax layer

model_2.add(Dense(5, activation='softmax'))

# Let us use the Adam optimiser 

optimiser = tf.keras.optimizers.Adam(learning_rate=0.001)
model_2.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_2.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_5 (Conv3D)           (None, 30, 120, 120, 8)   656       
                                                                 
 batch_normalization_5 (Batc  (None, 30, 120, 120, 8)  32        
 hNormalization)                                                 
                                                                 
 activation_5 (Activation)   (None, 30, 120, 120, 8)   0         
                                                                 
 conv3d_6 (Conv3D)           (None, 30, 120, 120, 16)  3472      
                                                                 
 activation_6 (Activation)   (None, 30, 120, 120, 16)  0         
                                                                 
 batch_normalization_6 (Batc  (None, 30, 120, 120, 16)  64       
 hNormalization)                                      

In [21]:
train_generator = generator_new(train_path, train_doc, batch_size)
val_generator = generator_new(val_path, val_doc, batch_size)

In [22]:
model_2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning:     `imread` is deprecated!
    `imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imread`` instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
    Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


Source path =  /content/drive/My Drive/Colab Notebooks/Project_data/train ; batch size = 40
Epoch 1/15
15/17 [=========================>....] - ETA: 12s - loss: 7.1389 - categorical_accuracy: 0.3233Batches:  17 Index: 40


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: DeprecationWarning:     `imread` is deprecated!
    `imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imread`` instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
    Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


17/17 [==============================] - ETA: 0s - loss: 6.8109 - categorical_accuracy: 0.3258Source path =  /content/drive/My Drive/Colab Notebooks/Project_data/val ; batch size = 40
Batches:  3 Index: 40

Epoch 1: saving model to model_init_2022-09-1119_46_19.528232/model-00001-6.81092-0.32579-1.38432-0.34000.h5
17/17 [==============================] - 132s 8s/step - loss: 6.8109 - categorical_accuracy: 0.3258 - val_loss: 1.3843 - val_categorical_accuracy: 0.3400 - lr: 0.0010
Epoch 2/15
17/17 [==============================] - ETA: 0s - loss: 2.8302 - categorical_accuracy: 0.4348
Epoch 2: saving model to model_init_2022-09-1119_46_19.528232/model-00002-2.83016-0.43478-2.57101-0.21667.h5
17/17 [==============================] - 74s 5s/step - loss: 2.8302 - categorical_accuracy: 0.4348 - val_loss: 2.5710 - val_categorical_accuracy: 0.2167 - lr: 0.0010
Epoch 3/15
17/17 [==============================] - ETA: 0s - loss: 2.0311 - categorical_accuracy: 0.4578
Epoch 3: saving model to model

**Conclusion (Image Size 140 X 140):** With a batch size of 40 and 15 epochs, the training accuracy (categorical_accuracy) is 0.70 and validation accuracy (val_categorical_accuracy) is 0.27

**Experiment 3:** CNN + LSTM w/Image Size of  160 X 160, batch size of 40 and 15 epochs

In [13]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation,Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D, Conv2D, MaxPooling2D
from keras.layers.recurrent import LSTM
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

In [14]:
x = 30 # number of frames
y = 160 # image width
z = 160 # image height

# Input all the images sequencial by building the layer with dropouts and batchnormalisation

model_3 = Sequential()   
model_3.add(TimeDistributed(Conv2D(16, (3, 3),padding='same', activation='relu'),input_shape=(x,y,z,3)))
model_3.add(TimeDistributed(BatchNormalization()))
model_3.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model_3.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu')))
model_3.add(TimeDistributed(BatchNormalization()))
model_3.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model_3.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
model_3.add(TimeDistributed(BatchNormalization()))
model_3.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model_3.add(TimeDistributed(Conv2D(128, (3, 3) , padding='same', activation='relu')))
model_3.add(TimeDistributed(BatchNormalization()))
model_3.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model_3.add(TimeDistributed(Conv2D(256, (3, 3) , padding='same', activation='relu')))
model_3.add(TimeDistributed(BatchNormalization()))
model_3.add(TimeDistributed(MaxPooling2D((2, 2))))

# Add a flatten layer 

model_3.add(TimeDistributed(Flatten()))

model_3.add(LSTM(64))
model_3.add(Dropout(0.25))

# Add a dense layer 
model_3.add(Dense(64,activation='relu'))
model_3.add(Dropout(0.25))

# Final Softmax layer

model_3.add(Dense(5, activation='softmax'))

# Adam optimiser

optimiser = tf.keras.optimizers.Adam(learning_rate=0.001) #write your optimizer
model_3.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_3.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, 30, 160, 160, 16)  448      
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, 30, 160, 160, 16)  64       
 tributed)                                                       
                                                                 
 time_distributed_2 (TimeDis  (None, 30, 80, 80, 16)   0         
 tributed)                                                       
                                                                 
 time_distributed_3 (TimeDis  (None, 30, 80, 80, 32)   4640      
 tributed)                                                       
                                                                 
 time_distributed_4 (TimeDis  (None, 30, 80, 80, 32)   1

In [15]:
#reducing batch size due to OOM error in the fit_generator step
batch_size = 16
train_generator = generator_new(train_path, train_doc, batch_size)
val_generator = generator_new(val_path, val_doc, batch_size)

In [20]:
model_3.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning:     `imread` is deprecated!
    `imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imread`` instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
    Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


Source path =  /content/drive/My Drive/Colab Notebooks/Project_data/train ; batch size = 16
Epoch 1/15
40/42 [===========================>..] - ETA: 6s - loss: 1.5055 - categorical_accuracy: 0.3453Batches:  42 Index: 16


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: DeprecationWarning:     `imread` is deprecated!
    `imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imread`` instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
    Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


42/42 [==============================] - ETA: 0s - loss: 1.5029 - categorical_accuracy: 0.3469Source path =  /content/drive/My Drive/Colab Notebooks/Project_data/val ; batch size = 16
Batches:  7 Index: 16

Epoch 1: saving model to model_init_2022-09-1122_01_23.945085/model-00001-1.50292-0.34691-1.76308-0.22000.h5
42/42 [==============================] - 164s 4s/step - loss: 1.5029 - categorical_accuracy: 0.3469 - val_loss: 1.7631 - val_categorical_accuracy: 0.2200 - lr: 0.0010
Epoch 2/15
42/42 [==============================] - ETA: 0s - loss: 1.3828 - categorical_accuracy: 0.4218
Epoch 2: saving model to model_init_2022-09-1122_01_23.945085/model-00002-1.38277-0.42177-2.10137-0.21429.h5
42/42 [==============================] - 66s 2s/step - loss: 1.3828 - categorical_accuracy: 0.4218 - val_loss: 2.1014 - val_categorical_accuracy: 0.2143 - lr: 0.0010
Epoch 3/15
42/42 [==============================] - ETA: 0s - loss: 1.4035 - categorical_accuracy: 0.3844
Epoch 3: saving model to model

**Conclusion (Image Size 160 X 160):** With a batch size of 16 and 15 epochs, the training accuracy (categorical_accuracy) is 0.56 and validation accuracy (val_categorical_accuracy) is 0.64